In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras import layers
from tensorflow.keras.layers import Activation, Dense, Flatten, BatchNormalization, Input, Lambda
from tensorflow.keras.optimizers import Adam, SGD, Adagrad, Adamax
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.metrics import binary_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.applications.vgg19 import VGG19
from keras.applications.vgg19 import preprocess_input
from sklearn.metrics import confusion_matrix
from keras.layers import Dropout
from tensorflow.keras import regularizers
from keras.models import Model
import itertools
import os
import shutil
import random
from glob import glob
import matplotlib.pyplot as plt
import warnings
import PIL
import cv2
%matplotlib inline

In [ ]:
import os
import cv2
import numpy as np

# Define the paths to your data folders
train_data_dir = '/content/drive/MyDrive/CLL_Train_More'
test_data_dir = '/content/drive/MyDrive/CLL_Test'

# Function to load and preprocess images
def load_and_preprocess_data(data_dir):
    # Initialize empty lists to store images and labels
    images = []
    labels = []

    # Iterate through the images in the directory
    for img_name in os.listdir(data_dir):
        # Read the image
        img = cv2.imread(os.path.join(data_dir, img_name))
        # Preprocess the image (e.g., resize, normalize)
        img = cv2.resize(img, (224, 224))  # Resize to fit your model's input size
        img = img.astype('float32') / 255.0  # Normalize pixel values to [0, 1]
        # Append the preprocessed image to the list
        images.append(img)
        # Append the label (1 for tumor)
        labels.append(1)

    return np.array(images), np.array(labels)

# Load and preprocess training data
train_images, train_labels = load_and_preprocess_data(train_data_dir)

# Load and preprocess test data
test_images, test_labels = load_and_preprocess_data(test_data_dir)


In [ ]:
from tensorflow.keras.utils import Sequence

class PatchDataGenerator(Sequence):
    def __init__(self, data_dir, patch_size, stride, batch_size):
        self.data_dir = data_dir
        self.patch_size = patch_size
        self.stride = stride
        self.batch_size = batch_size
        self.image_paths = [os.path.join(data_dir, fname) for fname in os.listdir(data_dir)]
        self.on_epoch_end()

    def __len__(self):
        return int(np.ceil(len(self.image_paths) * (512 // self.patch_size) ** 2 / self.batch_size))

    def __getitem__(self, index):
        batch_image_paths = self.image_paths[index * self.batch_size:(index + 1) * self.batch_size]
        X, y = self.__data_generation(batch_image_paths)
        return X, y

    def on_epoch_end(self):
        np.random.shuffle(self.image_paths)

    def __data_generation(self, batch_image_paths):
        images = []
        labels = []

        for img_path in batch_image_paths:
            img = cv2.imread(img_path)
            img = img.astype('float32') / 255.0
            patches = extract_patches(img, self.patch_size, self.stride)
            images.extend(patches)
            labels.extend([1] * len(patches))  # Assuming all images contain tumor

        return np.array(images), np.array(labels)

def extract_patches(image, patch_size, stride):
    patches = []
    h, w, _ = image.shape
    for i in range(0, h - patch_size + 1, stride):
        for j in range(0, w - patch_size + 1, stride):
            patch = image[i:i + patch_size, j:j + patch_size]
            patches.append(patch)
    return np.array(patches)

# Parameters
patch_size = 16
stride = 16
batch_size = 16  # Start with a smaller batch size

# Create data generators
train_generator = PatchDataGenerator(train_data_dir, patch_size, stride, batch_size)
test_generator = PatchDataGenerator(test_data_dir, patch_size, stride, batch_size)



In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, GlobalAveragePooling2D
from tensorflow.keras.applications import VGG16
from tensorflow.keras.optimizers import Adagrad
from tensorflow.keras.losses import BinaryCrossentropy

def create_deep_mil_model(input_shape):
    base = VGG16(weights='imagenet', include_top=False, input_shape=input_shape)

    #for layer in base.layers:
     #   layer.trainable = False


    x = base.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(128, activation='relu')(x)
    x = Dense(1, activation='sigmoid')(x)

    # Create an instance of the Adagrad optimizer with the desired learning rate
    optimizer = Adagrad(learning_rate=0.00002)

    # Compile the model with the custom optimizer
    model = Model(inputs=base.input, outputs=x)
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

    return model

# Update the input shape to match the size of your image patches
input_shape = (224, 224, 3)  # Patch size
deep_mil_model = create_deep_mil_model(input_shape)
deep_mil_model.summary()




58889256/58889256 [==============================] - 0s 0us/step
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                              

In [ ]:
# Train the Deep MIL model
history = deep_mil_model.fit(train_images, train_labels, epochs=3, batch_size=16)


Epoch 1/3
94/94 [==============================] - 44s 295ms/step - loss: 0.2797 - accuracy: 1.0000
Epoch 2/3
94/94 [==============================] - 19s 205ms/step - loss: 0.1825 - accuracy: 1.0000
Epoch 3/3
94/94 [==============================] - 20s 210ms/step - loss: 0.1278 - accuracy: 1.0000


In [ ]:
# Evaluate the model on the test set
test_loss, test_accuracy = deep_mil_model.evaluate(test_images, test_labels)

print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)


2/2 [==============================] - 11s 5s/step - loss: 0.1069 - accuracy: 1.0000
Test Loss: 0.10692152380943298
Test Accuracy: 1.0


In [ ]:

# Save the trained model to a file
deep_mil_model.save('/content/drive/MyDrive/trained_model_CLL.h5')


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
for i in range(len(test_images)):
    image = np.expand_dims(test_images[i], axis=0)
    confidence = deep_mil_model.predict(image)[0][0]
    print(f'Image {i + 1} - Tumor Confidence: {confidence * 100:.2f}%')


1/1 [==============================] - 1s 1s/step
Image 1 - Tumor Confidence: 90.85%
1/1 [==============================] - 0s 30ms/step
Image 2 - Tumor Confidence: 87.07%
1/1 [==============================] - 0s 28ms/step
Image 3 - Tumor Confidence: 90.83%
1/1 [==============================] - 0s 28ms/step
Image 4 - Tumor Confidence: 90.47%
1/1 [==============================] - 0s 25ms/step
Image 5 - Tumor Confidence: 91.15%
1/1 [==============================] - 0s 28ms/step
Image 6 - Tumor Confidence: 90.75%
1/1 [==============================] - 0s 32ms/step
Image 7 - Tumor Confidence: 89.31%
1/1 [==============================] - 0s 27ms/step
Image 8 - Tumor Confidence: 89.33%
1/1 [==============================] - 0s 26ms/step
Image 9 - Tumor Confidence: 90.73%
1/1 [==============================] - 0s 37ms/step
Image 10 - Tumor Confidence: 88.78%
1/1 [==============================] - 0s 34ms/step
Image 11 - Tumor Confidence: 90.73%
1/1 [==============================] - 0s 3

In [ ]:
TESTING THE MIL

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Load the trained model
model_path = '/content/drive/MyDrive/trained_model_CLL.h5'
trained_model = tf.keras.models.load_model(model_path)

def preprocess_image(image_path):
    # Load and preprocess the test image
    img = cv2.imread(image_path)
    img = cv2.resize(img, (224, 224))
    img = img.astype('float32') / 255.0
    return img

def generate_heatmap(model, image):
    # Generate the heatmap for the test image
    grad_model = tf.keras.models.Model([model.inputs], [model.get_layer('block5_conv3').output, model.output])

    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model(image)
        top_pred_index = tf.argmax(predictions[0])
        top_class_channel = predictions[:, top_pred_index]

    grads = tape.gradient(top_class_channel, conv_outputs)[0]
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1))

    conv_outputs = conv_outputs[0].numpy()
    pooled_grads = pooled_grads.numpy()

    for i in range(pooled_grads.shape[-1]):
        conv_outputs[:, :, i] *= pooled_grads[i]

    heatmap = np.mean(conv_outputs, axis=-1)
    heatmap = np.maximum(heatmap, 0) / np.max(heatmap)

    return heatmap, predictions[0][top_pred_index] * 100  # Extracting tumor confidence

# Path to the new external test image
test_image_path = '/content/drive/MyDrive/MIL_CLL_TEST/30.png'

# Preprocess the test image
test_image = preprocess_image(test_image_path)

# Generate the heatmap for the test image
heatmap, tumor_confidence = generate_heatmap(trained_model, np.expand_dims(test_image, axis=0))

# Adjust the brightness of the heatmap
heatmap = cv2.resize(heatmap, (test_image.shape[1], test_image.shape[0]))
heatmap = np.uint8(255 * heatmap * 0.5)  # Adjust brightness by multiplying by 0.5

# Display the original test image and the heatmap
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
plt.imshow(test_image)
plt.title('Original Test Image')
plt.axis('off')

plt.subplot(1, 2, 2)
plt.imshow(heatmap, cmap='jet')
plt.title('Heatmap - Tumor Confidence: {:.2f}%'.format(tumor_confidence))
plt.colorbar()
plt.axis('off')

plt.show()
